<a href="https://colab.research.google.com/github/roncv/MPss/blob/main/filament_mp_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MPss - Filament & Membrane Protein

Protocol for associatiation of protofilaments to a lipid bilayer with an embedded (trans)membrane protein.

The protein structures are converted to  [Martini 3](https://www.nature.com/articles/s41592-021-01098-3) coarse-grained (CG) representations using [Martinize](https://github.com/marrink-lab/vermouth-martinize). The membrane-spanning regions and protein orientation is predicted using [Memembed](https://doi.org/10.1186/1471-2105-14-276). A lipid bilayer is constructed using [Insane](https://doi.org/10.1021/acs.jctc.5b00209). The system is briefly equilibrated using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).

Change the settings below then click **Runtime → Run all**.  You will be prompted to upload a Soluble Protein .pdb file to simulate.

**NB:** To generate filaments the .pdb file supplied should contain crystal lattice vectors. 

In [ ]:
#@title Define Settings
#@markdown #### Set Membrane Type:
MembraneType = "POPE:POPG:CARDIOLIPIN" #@param ["POPC","POPE:POPG","POPE:POPG:CARDIOLIPIN"]
PE_PG_ratio = 3.5 #@param {type:"slider", min:1, max:10, step:0.1}
CARD_conc = 0.05 #@param {type:"slider", min:0.0, max:0.3, step:0.01}
if CARD_conc == 0.0:
    card = False
#@markdown ---
#@markdown #### Define Orientation of the Membrane Protein:
Nterminus = "in" #@param ["in","out"]
#@markdown ---
#@markdown #### Set number of Repeats
Repeats = 5 #@param {type:"slider", min:1, max:25, step:1}
#@markdown ---
#@markdown #### Rotate Protein:
Rotate = False #@param {type:"boolean"}

#@markdown ---
#@markdown #### Set Box Dimensions:
Box_Width = 30 #@param {type:"slider", min:8, max:30, step:1}
Box_Height = 22 #@param {type:"slider", min:12, max:30, step:1}
#@markdown #### Add padding to the ends of filaments:
Box_Length_ext = 10 #@param {type:"slider", min:0, max:20, step:1}
#@markdown ---
#@markdown #### Create Filament:
Filament = True #@param {type:"boolean"}
Units = 4 #@param {type:"slider", min:1, max:20, step:1}
#Continuous = False #@param {type:"boolean"}
#@markdown ---
#@markdown #### Define Distance between Protein and Membrane (nm):
Embed = 2 #@param {type:"slider", min:-3, max:4, step:0.5}
#@markdown ---
#@markdown #### Output:
Experiment_Name = "fil_mp" #@param {type:"string"}
Download = True #@param {type:"boolean"}
Zip_Name = "fil_mp" #@param {type:"string"}

In [ ]:
#@title Upload Protein Structure
# Upload protein .pdb structure

import glob
import shutil
import subprocess
from random import randrange
import os
import sys
from decimal import *
import random

!python3 -m pip install py3dmol
!python3 -m pip install colorama

import py3Dmol
from colorama import Fore
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

def upload_proteins(dirname='filament_mp'):
    """
    ...
    """
    os.chdir('/content/')
    print(Fore.BLUE + "\nUpload Membrane Protein Structure:\n")
    upload1 = files.upload() # request upload of .pdb - membrane

    print(Fore.BLUE + "\nUpload Soluble Protein Structure:\n")
    upload2 = files.upload() # request upload of .pdb - soluble

    # Arrange working directories
    filename1 = next(iter(upload1))
    name1 = os.path.splitext(filename1)[0]
    filename2 = next(iter(upload2))
    name2 = os.path.splitext(filename2)[0]

    name = f"{name1}-{name2}-{dirname}" # combine names
    working_dir = f'/content/{name}/'

    if os.path.exists(working_dir):
        shutil.rmtree(working_dir)
    os.makedirs(working_dir)

    # Protein 1 - Membrane
    print(Fore.GREEN + "\nMembrane Protein:\n")
    os.rename(filename1, working_dir + filename1)
    os.chdir(working_dir)
    mol1 = open(working_dir + filename1, 'r').read()
    mview = py3Dmol.view(width=500,height=280)
    mview.addModel(mol1,'pdb')
    mview.setStyle({'cartoon':{'color':'spectrum'}})
    mview.setBackgroundColor('0xffffff')
    mview.zoomTo()
    mview.show()

    # Protein 2 - Soluble
    print(Fore.GREEN + "\nSoluble Protein (Filament):\n")
    os.chdir('/content/')
    os.rename(filename2, working_dir + filename2)
    os.chdir(working_dir)
    mol2 = open(working_dir + filename2, 'r').read()
    mview = py3Dmol.view(width=500,height=280)
    mview.addModel(mol2,'pdb')
    mview.setStyle({'cartoon':{'color':'spectrum'}})
    mview.setBackgroundColor('0xffffff')
    mview.zoomTo()
    mview.show()

    return working_dir, name, filename1, filename2

working_dir, name, filename1, filename2 = upload_proteins(dirname=Experiment_Name) # usage

# Directory organsiation...
os.chdir(working_dir)
for file in glob.glob(r'/content/martini*.itp'): # martini .itp files
    print(file)
    shutil.copy(file, working_dir)

In [ ]:
#@title Install Dependencies
# Install Dependencies
%%capture
os.chdir('/content/')
if not os.path.isdir("/content/memembed/"):
  !apt-get update -y
  !apt-get install dssp
  !git clone https://github.com/timnugent/memembed
  %cd memembed
  !make
  %cd ../
  !python3 -m pip install pdb2pqr
  !python3 -m pip install vermouth
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis
  !git clone https://github.com/pstansfeld/cg2at
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/martini_v300.zip
  !unzip -o martini_v300.zip
  !wget 'http://cgmartini.nl/images/applications/water/water.gro'
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/insane3.py
  !wget https://github.com/pstansfeld/MemProtMD/raw/main/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx

import gromacs
import MDAnalysis
from MDAnalysis import *
from MDAnalysis.lib.distances import self_capped_distance
from MDAnalysis.lib.distances import capped_distance
import numpy as np
from numpy import *
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
#@title Get lipid concentrations
# Function to get clean ratio of lipids for a given concentration of CARD
def get_lipid_ratio(pe_pg_ratio=3.5, pg_conc=None, total=10, card_conc=0.1, tolerance=0.1):
    """
    Function to get clean a ratio of lipids for a given concentration of cardiolipin.

    Parameters
    
    pe_pg_ratio : Ratio of PE:PG in PE:PG or PE:PG:CARD membrane.
    pg_conc : Float between 0 and 1. Conc. of pg within a PE:PG only membrane. Only takes 2.d.p.
    total : Target ratio sum total. May exceed depending on error tolerance.
    card_conc : Concentration of CARD in a PE:PG:PE membrane.
    tolerance : Error tolerance for ratio.
    """
    
    # No CARD
    if card_conc == 0:
        if pg_conc is None:
            pg_conc = 1 - (pe_pg_ratio / (pe_pg_ratio+1))
            #print(f"PG conc: {round(pg_conc, 2)}")
        elif pg_conc == 0:
            #print(f"PG conc: {round(pg_conc, 2)}")
            ratio = [1, 0, 0]
            return ratio

        #if tolerance == 0: # maybe ignore this when conc is given
        pg_conc = Decimal(f"{pg_conc}")
        pg_conc = round(pg_conc, 2)
        #print(f"PG conc: {pg_conc}")
        pg_int = int(pg_conc * 100)
        pe_int = 100 - pg_int
        pe_pg_ratio = round(pe_int/pg_int, 1)
        #print(f"PE:PG ratio - {pe_pg_ratio} : 1")
        ratio = [pe_int, pg_int, 0]

        # Use HCF, LCM, GCD ¿to utilise totals? and minimise this value ***
        # https://softwareengineering.stackexchange.com/questions/416627/how-to-find-the-nearest-integer-number-that-when-multiplied-by-a-set-of-decimals
    
    # With CARD
    else:
        card_conc = Decimal(f"{card_conc}")

        # CARD concentration
        card_flt = card_conc * total # float result
        card_rnd = int(round(card_conc * total, 0)) # rounded result
        card_err = np.sqrt(np.square(card_flt-card_rnd)) # difference

        # Fit to tolerance
        r = 0 # counter
        if float(card_err) <= tolerance:
            pass
        else:
            while float(card_err) > tolerance:
                total+=1; r+=1 # counters
                card_flt = card_conc * total
                card_rnd = int(round(card_conc * total, 0))
                card_err = np.sqrt(np.square(card_flt-card_rnd))
        # Set CARD count
        card = card_rnd

        # PE + PG conc
        pe = int(2*(total//(2*(pe_pg_ratio+1)))*pe_pg_ratio)
        pg = total - (pe+card_rnd)
        ratio = [pe, pg, card]

    return ratio

#ratio = get_lipid_ratio(pe_pg_ratio=PE_PG_ratio, total=10, card_conc=CARD_conc, tolerance=0.01) # usage
#ratio = get_lipid_ratio(pe_pg_ratio=3.5, pg_conc=0.3, total=10, card_conc=0) # usage
#ratio

In [ ]:
#@title Functions
# title Get area per lipid
def get_area_per_lipid(ratio):
    """
    Function to get the area per lipid for a given lipid ratio.
    """
    # Constants - area per lipid
    pe_apl = 0.63
    pg_apl = 0.67
    cl_apl = 1.23

    # Weighted densities
    pe_wd = ratio[0]/sum(ratio)*pe_apl
    pg_wd = ratio[1]/sum(ratio)*pg_apl
    cl_wd = ratio[2]/sum(ratio)*cl_apl

    # Sum up for avg. density
    avg_apl = sum([pe_wd,pg_wd,cl_wd])

    return avg_apl

#get_area_per_lipid(ratio) # usage


# ANSI Escape Code styling
def esc(codes, string:str):
    """
    Function to generate ANSI escape codes for customizing cell output.
    """
    if isinstance(codes, list): # process list format
        # Generate text with input string, then format as ANSI escape code
        return f"\033[{';'.join(map(str, codes))}m{string}\033[0m"
    if isinstance(codes, int): # process as an integer
        return f'\033[{codes}m{string}\033[0m'
    else:
        raise TypeError("ANSI codes must be provided as a list or integer")


# Get Cell Dimensions
def get_dimensions(structure):
    """
    Function to get dimensions of a system's cell from a .pdb file.
    """
    # Create universe with structure file...
    u = MDAnalysis.Universe(structure)
    x = round(float(u.dimensions[0]/10)) # in angstroms
    y = round(float(u.dimensions[1]/10))
    z = round(float(u.dimensions[2]/10))
    cube = [x,y,z]

    return x, y, z, cube
    
#x, y, z, cube = get_dimensions('CG-system-extended.gro') # usage


# Format topology file
def format_topology(topology, Units=1):
    """
    Format a topology file to include all .itp files.
    """
    replacements = {'Protein        1\n':f'ProteinA      {Units}' +'\n','NA+':'NA',
                    'CL-':'CL',
                    '#include "martini_v3.itp"':'#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n'}
    lines = []
    with open(topology) as infile:
        for line in infile:
            for src, target in replacements.items():
                line = line.replace(src, target)
            lines.append(line)
    with open(topology, 'w') as outfile:
        for line in lines:
            outfile.write(line)

#format_topology('topol_test2.top') # usage


# Position protein
def position_protein_insane(structure, selection='name BB or name SC*',
                            monolayer_width=1.95):
    """
    Finds the COM and minimum z-coordinate of protein structure given. Difference 
    in values is added to the average monolayer width. Made to work with the insane3.py 
    -center and -dm flags, where the functions output used as the -dm value results in 
    the protein just touching the membrane. To embed the protein more or less, increase 
    or decrease the value accordingly.

    Distances in nm.
    """
    # Create universe
    u = MDAnalysis.Universe(structure)

    # Protein dimensions in z-axis
    z_coords_p = u.select_atoms(f'{selection}').positions[:, 2] # select protein
    p_min_z = float(z_coords_p.min())/10.0
    com_p = (u.select_atoms(f'{selection}').center_of_mass()/10.0)[2]
    adjustment = (com_p - p_min_z) + monolayer_width # in nm

    return adjustment

#position_protein_insane('protein-em.pdb') # usage


# Topology file entries
def insane_topadd(top_add, og_top):
    """
    Function to add new entries to a topology file.
    """
    with open(og_top, "a") as output_file:
        with open(top_add) as input_data:
            # Find additional entries for topology
            for line in input_data:
                if line.strip() == 'Protein        1':
                    break
            for line in input_data:  # read + write them to original file
                output_file.write(line.rstrip('\n').strip()+"\n")

#insane_topadd('topol_ext.top', 'topol1.top') # usage


# Interrupt cell execution at specified opint
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
# Function call for cell interruption
def exit_c(): raise StopExecution

#exit_c() # usage


# Write EM mdp file
def mdp_em(filename='em'):
    """
    Write an .mdp file for EM
    """
    with open(f'{filename}.mdp','w') as em:
                em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')

#mdp_em() # usage


# Format .pdb
def format_pdb_residues(pdb):
    """
    Format residues of a .pdb.
    """
    with open(pdb, 'r') as file :
        filedata = file.read()
    filedata = filedata.replace('HSE', 'HIS')
    filedata = filedata.replace('HSD', 'HIS')
    filedata = filedata.replace('MSE', 'MET')
    filedata = filedata.replace(' SE ', ' SD ')
    
    # Update .pdb
    with open(pdb, 'w') as file:
        file.write(filedata)

#format_pdb_residues(pdb='protein1.pdb') # usage


# Write a topology file
def write_topology(idx, idx_a, n=1):
    """
    Write Topology file for protein.
    """
    with open('topol.top','w') as top:
                top.write('#include "martini_v3.0.0.itp"\n'
                            '#include "martini_v3.0.0_ions_v1.itp"\n'
                            '#include "martini_v3.0.0_solvents_v1.itp"\n'
                            '#include "martini_v3.0.0_phospholipids_v1.itp"\n'
                            f'#include "protein{idx}-cg.itp"\n[ system ]\n[ molecules ]\nProtein{idx_a} {n}')

    os.system(f'cp topol.top topol_protein{idx_a}.top')

#write_topology(idx=1, idx_a='A') # usage


# Visualise System
def visualise_system(file, _type='gro', waters=False, ions=False, dir=working_dir):
    """
    Visualise systems in 3D using Py3Dmol.
    """
    if file is not None: # specified filepath (rel/full)
        systems = [file]
        _type = file[-3:] # get file extension
    else:
        # Access first repeat of each experiment - automated
        if _type == 'pdb':
            systems = glob.glob(os.path.expanduser(f'{dir}*/CG-system1.{_type}'))
        else:
            systems = glob.glob(os.path.expanduser(f'{dir}*/EQ1/eq.{_type}'))
    # Visualise
    for i in systems:
        print(f'{Fore.BLUE}\n{i}\n') # convert to f strings ***
        mview = py3Dmol.view(width=600,height=450)
        mol1 = open(i, 'r').read()
        mview.addModel(mol1,f'{_type}')
        mview.setStyle({'cartoon':{'color':'spectrum'}})
        mview.setStyle({'atom':'PO1','atom':'PO2','atom':'PO3','atom':'PO4'},
                       {'sphere':{'color':'0xE63629', 'radius':2.2,'quality':20}}) # lipids
        if waters is True:
            mview.setStyle({'atom':'W'},{'sphere':{'color':'0x5014F5','quality':20}}) # waters
        if ions is True:
                try:
                    mview.setStyle({'atom':'CL'},{'sphere':{'color':'0x00E649','radius':1.5,'quality':20}}) # CL
                except:
                    pass
                try:
                    mview.setStyle({'atom':'NA'},{'sphere':{'color':'0xE644D5','radius':1.5,'quality':20}}) # NA
                except:
                    pass
        mview.setStyle({'atom':'BB'},
                       {'sphere':{'color':'0x5FC8F5', 'radius':3,'quality':20}}) # protein
        try:
            mview.setStyle({'chain':'B', 'atom':'BB'},
                           {'sphere':{'color':'0x17E6DB', 'radius':3,'quality':20}}) # protein
        except:
            pass
        mview.setBackgroundColor('0xffffff')
        mview.addUnitCell() # only with .pdb files
        mview.zoomTo()
        mview.show()
        
        # Remove temporary files
        for file in glob.glob(r'#*'):
            os.remove(file)
    # Change back to working directory
    #os.chdir(dir)
    # Remove temporary files
    #for file in glob.glob(r'*/#*'):
    #    os.remove(file)
    
#visualise_system('pdb', waters=False, ions=False) # use .pdb to see unit cell

In [ ]:
#@title Predict Membrane Protein Orientations
def predict_mp_orientation(Nterminus='in', structure=filename1, o='memembed1', dir=working_dir):
    """
    ...
    """
    os.chdir(dir)
    os.system(f'/content/memembed/bin/memembed -n {Nterminus} -o {o}.pdb {structure}')
    print(f"{esc(91, f'periplasmic')}\n{esc(91, f'oooo   oooo')}\n¦¦¦¦ {esc([97,100], f' ')} ¦¦¦¦\n{esc(34, f'oooo   oooo')}\n{esc(34, f'cytoplasmic')}\n")
    #mview = py3Dmol.view(width=800,height=400)  
    #mol1 = open(f'{dir}/{o}.pdb', 'r').read()
    #mview.addModel(mol1)
    #mview.setStyle({'cartoon':{'color':'spectrum','quality':20}})
    #mview.setStyle({'resn':'DUM'},{'sphere':{'radius':0.4,'quality':20}})
    #mview.setBackgroundColor('0xffffff')
    #mview.zoomTo()
    #mview.show()

#predict_mp_orientation(Nterminus=Nterminus, structure=filename1) # usage

# 
def position_membrane_protein(mem_prot, sol_prot, idx=1, Box_Width=Box_Width,
                              Box_Height=Box_Width):
    """
    ...
    """
    cube = [Box_Width,Box_Width,Box_Height]

    gromacs.make_ndx(f=mem_prot, o=f'index{idx}.ndx', input=('del 0', 'del 1-100','rDUM','q'),backup=False)
    gromacs.editconf(f=mem_prot,o=f'centered{idx}.pdb',n=f'index{idx}.ndx',c=True,box=cube,input=(0,0),backup=False)

    x, y, z, cube = get_dimensions(f'centered{idx}.pdb')

    gromacs.confrms(f2=mem_prot, f1=f'centered{idx}.pdb', one=True, o=f'aligned{idx}.gro', input=(3,3),backup=False)
    gromacs.editconf(f=f'aligned{idx}.gro', o=f'protein{idx}.pdb',label='A',resnr=1,n=f'index{idx}.ndx',box=cube,input=(0,0),backup=False)

    v = MDAnalysis.Universe(f'aligned{idx}.gro')
    dum = v.select_atoms('resname DUM')

    #print("\n", [x, y, z], "\n")
    dm = (z/2) - (round(dum.center_of_mass()[2]/10))

    return dm, cube

#dm, cube = position_membrane_protein(mem_prot='memembed1.pdb', sol_prot=filename2, idx=1) # usage

In [ ]:
#@title Build Systems
def coarse_grain_protein(idx, idx_a, ElasticCutoff=0.7,cube=None,rotation=None,translation=None, Continuous=False, dir=working_dir): # cube=cube - clarity?
    # martini .itp files
    for file in glob.glob(r'/content/martini_v300/*.itp'):
        #print(file)
        shutil.copy(file, dir)
    
    # GENERATE COARSE GRAINED PROTEIN
    print(f"\nCoarse graining protein {idx}({idx_a})...")
    merge = 'A' 
    eunit = 'molecule' # 'molecule
    os.system(f'martinize2 -f protein{idx}.pdb -dssp mkdssp -ff martini3001 '
              f'-x protein{idx}-cg.pdb -o protein{idx}-cg.top -elastic -ef 500 -eu {str(ElasticCutoff)} ' 
              f'-el 0.5 -ea 0 -ep 0 -merge {merge} -maxwarn 100000 -eunit {eunit}') # see eunit + merge (chains...)
    print("Done.")

    os.system(f"sed -e 's/^molecule.*/Protein{idx_a} 1/g' molecule_0.itp | grep -v ';' >  protein{idx}-cg.itp")

    # Continuity restraints - add continuity to filament elastic network 
    if Continuous == True:
        u = MDAnalysis.Universe(f'protein{idx}-cg.pdb')
        boxed = u.dimensions

        with open(f"protein{idx}-cg.itp") as f:
            contents1, constraints, contents2 = f.read().partition("[ constraints ]\n")

        # Restraints - add continuity to filament elastic network
        U = MDAnalysis.Universe(f'protein{idx}-cg.pdb')
        BB = U.select_atoms('name BB')
        d1 = pd.DataFrame(np.column_stack(self_capped_distance(BB.positions,max_cutoff=7,min_cutoff=5,box=boxed,method='pkdtree')))
        d2 = pd.DataFrame(np.column_stack(self_capped_distance(BB.positions,max_cutoff=7,min_cutoff=5,method='pkdtree')))
        zid = pd.DataFrame(BB.ix+1)
        d1 = d1.sort_values(by=[0, 1])
        d2 = d2.sort_values(by=[0, 1])
        df = pd.concat([d1,d2])
        df = df.sort_values(by=[0, 1])
        df = df.drop_duplicates(keep=False,subset=(0,1))
        df[2] = (df[2]/10).map('{:.5f}'.format)
        zi = dict(zip(zid.index,zid[0]))
        df[3] = df[0].replace(zi).map('{:.0f}'.format)
        df[4] = df[1].replace(zi).map('{:.0f}'.format)
        df[5] = 1
        df[6] = 500
        df = df[df[0]<df[1]]
        infinity = df.to_csv(columns=(3, 4, 5, 2, 6),sep='\t', index=False, header=False)    

        # New .itp parameters
        with open(f"protein{idx}-cg.itp", "w") as new:
            new.write(contents1 + infinity + constraints + contents2)

    #print("\nWriting files...")
    file_object = open(f'protein{idx}-cg.itp', 'a')
    file_object.write(f'#ifdef POSRES\n#include "protein{idx}-posre.itp"\n#endif\n')
    file_object.close()

    #print("\nRestraints...") # use of this??
    gromacs.genrestr(f=f'protein{idx}-cg.pdb',o=f'protein{idx}-posre.itp',input=(1,1)) # **??
    
    # EM .mdp 
    mdp_em()
    os.system(f'cp em.mdp em{idx}.mdp')

    # Setup and run minimisation
    #print("\nTidy file system...")
    try:
        os.remove(f'em{idx}*')
    except:
        pass

    if rotation is not None:
        #print("\nSingle condition...")
        gromacs.editconf(f=f'protein{idx}-cg.pdb', o=f'protein{idx}-box.pdb', c=True, box=cube, rotate=rotation)
    else:
        #print("\nNone condition...")
        gromacs.editconf(f=f'protein{idx}-cg.pdb', o=f'protein{idx}-box.pdb', c=True, box=cube)
    print(f"\nMinimizing protein {idx}({idx_a})...")
    gromacs.grompp(f=f'em{idx}.mdp',o=f'em{idx}.tpr',c=f'protein{idx}-box.pdb',maxwarn='-1',backup=False,v=True)
    gromacs.mdrun(deffnm=f'em{idx}', c=f'protein{idx}-em.pdb',backup=False)
    if translation is not None:
        gromacs.editconf(f=f'protein{idx}-em.pdb', o=f'protein{idx}-em.pdb', translate=translation)
    print("Done.\n")

#coarse_grain_protein(idx=1, idx_a='A') # usage

def build_proteins(mp, sp, Units=4, rotate_sp=180, Box_Width=Box_Width,
                   Box_Height=Box_Height, Box_Length_ext=None, Embed=0,
                   Continuous=False, Filament=Filament, dir=working_dir):
    """
    Parameters:

    mp: str, path object or file-like object
    Membrane protein

    sp: str, path object or file-like object
    Soluble protein

    Units: int
    Number of units in the protofilament

    rotate_sp: int
    Rotate filament (sp), 0 for no rotation
    """

    # Predict membrane orientation of input mp
    predict_mp_orientation(Nterminus=Nterminus, structure=mp, o='memembed1', dir=dir) # usage - red is periplasmic

    # Get membrane positining
    dm, cube = position_membrane_protein(mem_prot='memembed1.pdb', sol_prot=sp, idx=1)

    # Format mp residues
    format_pdb_residues(pdb='protein1.pdb')

    # Write mp topology
    write_topology(idx=1, idx_a='A')

    # Convert mp to CG rep
    coarse_grain_protein(idx=1, idx_a='A', cube=cube, dir=dir)

    # Generate protofilaments from soluble protein
    if Filament == True:
        gromacs.genconf(f=sp,o='ready2.pdb',nbox=[Units,1,1],backup=False,renumber=True)
        x, y, z, _ = get_dimensions('ready2.pdb')
        if Box_Length_ext is not None:# or Box_Length_ext != 0:
            cube = [x+Box_Length_ext, Box_Width, Box_Height] # add x-padding
            Continuous = False # prevent continuous filament with x-padding
        else:
            cube = [x, Box_Width, Box_Height]
        gromacs.editconf(f='protein1-em.pdb',o='protein1-em.pdb',c=True,box=cube) # format membrane protein cell size
    else:
        gromacs.genconf(f=sp,o='ready2.pdb',nbox=[1,1,1],backup=False,renumber=True)  
        cube = [Box_Width,Box_Width,Box_Height] 

    print(f"\nDimensions: {cube}")
    
    # Make .ndx for filament
    gromacs.make_ndx(f='ready2.pdb', o='index2.ndx',input=('del 0','del 1-100','rDUM','q'),backup=False)
    # Center in defined box size
    gromacs.editconf(f='ready2.pdb',o='protein2.pdb',n='index2.ndx',c=True,box=cube,input=(0,0),backup=False,label='A',resnr=1)
    format_pdb_residues(pdb='protein2.pdb') # format residues
    write_topology(idx=2, idx_a='B') # write filament .top file

    # Convert protofilament to CG rep + rotate if necessary
    coarse_grain_protein(idx=2,idx_a='B',rotation=[rotate_sp,0,0],
                         cube=cube,dir=dir, Continuous=Continuous)
    
    # Position soluble protein in box
    p_height = position_protein_insane('protein2-em.pdb')
    Distance = Embed + p_height
    print(f'SP Distance: {Distance}\n')
    gromacs.editconf(f=f'protein2-em.pdb', o=f'protein2-em.pdb', translate=[0,0,-Distance])

    # Match cell sizes for both proteins
    u = MDAnalysis.Universe('protein2-em.pdb')
    cube = list((u.dimensions[:3]/10)) # get cell dimensions of EM'd filament
    gromacs.editconf(f='protein1-em.pdb', o='protein1-em.pdb', c=True, box=cube) # apply dimensions to mp

    return

#build_proteins(mp=filename1, sp=filename2, Units=Units, Continuous=False, Box_Length_ext=Box_Length_ext) # usage

In [ ]:
#@title Equillibrate Systems
# %%script false --no-raise-error
def setup_cg_system(mp, sp, Units=4, rotate_sp=180, rotate_mp=True,
                    MembraneType=MembraneType, Repeats=Repeats, ElasticCutoff=0.7,
                    NamingConvention=None, Box_Width=Box_Width, Box_Height=Box_Height,
                    Box_Length_ext=None, Embed=0, Continuous=Continuous,
                    Ratio=None, dir=working_dir):
    """
    Function to set-up coarse-grained membrane-protein systems.
    """
    # rotate mp
    # translate mp

    print(f'\n{esc([100, 97], "INITIALIZE")}\n')

    # Move to working directory
    os.chdir(dir)

    # Options
    if NamingConvention == "elastic_cutoff": # ***
        name_nc = f'-eu{ElasticCutoff}'
    elif NamingConvention == None:
        name_nc = ''
    else:
        name_nc = NamingConvention

    # LIPID COMPOSITION
    if isinstance(Ratio, list):
        pe, pg, crd = Ratio
        if crd == 0:
            lipid_name = f'pe{pe}-pg{pg}'
            lipid = f'-l POPE:{pe} -l POPG:{pg}'
        else:
            lipid_name = f'pe{pe}-pg{pg}-c{crd}'
            lipid = f'-l POPE:{pe} -l POPG:{pg} -l CARD:{crd}'
        apl = get_area_per_lipid(Ratio)
        print(f"Lipid Ratio: {Ratio}")
    else:
        raise TypeError("Invalid lipid selection!")

    # Organize Directories - Including naming conventions
    name_filaments = f"f{Units}"
    name_embed = f"eb{Embed}"
    subdir = f"{dir}{name}-{lipid_name}-{name_filaments}-{name_embed}{name_nc}/"
    print(subdir)
    shutil.rmtree(subdir, ignore_errors=True)
    os.makedirs(subdir, exist_ok=True)
    shutil.copyfile(mp, subdir+mp)
    shutil.copyfile(sp, subdir+sp)
    os.chdir(subdir)

    # BUILD PROTEINS
    print("\nBuilding Proteins...\n")
    build_proteins(mp=mp,sp=sp,Units=Units,Continuous=Continuous,Embed=Embed,
                   Box_Length_ext=Box_Length_ext,rotate_sp=rotate_sp,Box_Width=Box_Width,
                   Box_Height=Box_Height,Filament=Filament, dir=subdir) # usage

    # CONTINUITY
    # .mdp settings
    if Continuous == True:
        pcoupltype = 'anisotropic'
        compressibility = '3e-4 3e-4 3e-4 0 0 0'
        ref_p = '1.0 1.0 1.0 0 0 0'
        # New .itp parameters ** not yet functional
        #with open("protein-cg.itp", "w") as new:
        #    new.write(contents1 + infinity + constraints + contents2)
    else:
        pcoupltype = 'semiisotropic'
        compressibility = '3e-4 3e-4'
        ref_p = '1.0 1.0'

    # REPEATS
    for rep in range(1,Repeats+1):
        #rep=str(rep)

        print(f"\n{esc(1, f'Repeat {rep}')} ({esc(34, f'CG-system{rep}')})")
        os.chdir(subdir)

        # Random rotation
        if rotate_mp is True:
            mp_rotation = random.randint(0,360)
            print(f"\nRotation {rep}: {mp_rotation}")
            gromacs.editconf(f=f'protein1-em.pdb', o=f'protein1-em{rep}.pdb',
                             c=True,rotate=[0,0,mp_rotation])
            
        # Random translation ***
        # ...

        # Add lipids
        x, y, z, cube = get_dimensions('protein1-em.pdb')
        dm = 0 # mp already centered (mmembrane centered also)
        print("\nAdding lipids...")
        print(f"Lipid ratio: {Ratio}")
        print(f"Area per lipid: {round(apl, 3)}")
        os.system(f'python2.7 /content/insane3.py {lipid} -a {apl} '
                  f'-o protein1-system.gro -p topol.top -f protein1-em{rep}.pdb '
                  f'-center -ring -x {x} -y {y} -z {z} -dm {dm}')
        print("Done.\n")

        #os.system("wget 'http://cgmartini.nl/images/applications/water/water.gro'") # remove ***

        # COMBINE PROTEINS + BILAYER
        u1 = MDAnalysis.Universe('protein1-system.gro')
        u2 = MDAnalysis.Universe("protein2-em.pdb")
        multi = MDAnalysis.Merge(u2.atoms, u1.atoms)
        multi.atoms.write("protein12.pdb")
        
        # Format topology
        replacements = {'Protein        1\n':'ProteinB       1\nProteinA       1\n',
                        '#include "protein-cg.itp"':'#include "protein1-cg.itp"\n#include "protein2-cg.itp"\n',
                        'NA+':'NA', 'CL-':'CL',
                        '#include "martini_v3.itp"':'#include "martini_v3.0.0.itp"\n#include "martini_v3.0.0_ions_v1.itp"\n#include "martini_v3.0.0_solvents_v1.itp"\n#include "martini_v3.0.0_phospholipids_v1.itp"\n'}
        lines = []

        with open('topol.top') as infile:
            for line in infile:
                for src, target in replacements.items():
                    line = line.replace(src, target)
                lines.append(line)
        with open('topol.top', 'w') as outfile:
            for line in lines:
                outfile.write(line)
        
        # SOLVATION
        #x, y, z, cube = get_dimensions('protein12.pdb')
        print("\nSolvating the system...")
        os.system(f'python2.7 /content/insane3.py -sol W -salt 0.15 '
                  f'-o CG-system{rep}.gro -p topol_solv_test.top -f protein12.pdb '
                  f'-center -ring -x {x} -y {y} -z {z}')
        # Update topology
        insane_topadd('topol_solv_test.top', 'topol.top') 
        os.system(f'cp topol.top topol{rep}.top')
        print("Done.")
        
        # ENERGY MINIMIZATION
        print(f"\nPreparing CG system {rep} for energy minimization...")
        mdp_em() # generate mdp
        gromacs.grompp(f='em.mdp',o=f'em_m{rep}.tpr',c=f'CG-system{rep}.gro',maxwarn='-1',backup=False,v=True) # em naming ***
        gromacs.mdrun(deffnm=f'em_m{rep}', c=f'CG-system{rep}.pdb',backup=False)
        gromacs.trjconv(f=f'CG-system{rep}.pdb', o=f'CG-system{rep}.pdb', pbc='mol',
                        s=f'em_m{rep}.tpr', conect=True, input=[0,0],backup=False)
        os.makedirs(f'EQ{rep}', exist_ok=True)
        #gromacs.make_ndx(f='CG-system'+ rep +'.pdb', o='index_sys'+ rep +'.ndx', input=('del 0', 'del 1-40', '0|rPOP*','1&!0','!1','del 1','name 1 LIPID','name 2 SOL_ION','q'),backup=False)
        gromacs.make_ndx(f=f'CG-system{rep}.pdb', o=f'index_sys{rep}.ndx',
                         input=('del 2-100', 'rPOP*|rCAR*','rW*|rION','name 1 PROTEIN',
                                'name 2 LIPID','name 3 SOL_ION','q'),
                         backup=False)
        print("System minimzed.")

        # EQUILIBRATION
        print(f"\nPreparing CG system {rep} for equillibration...")
        with open('cgequil.mdp','w') as md: # create equil .mdp
            md.write('define = -DPOSRES\nintegrator = md\ntinit = 0.0\ndt = 0.02\n'
                     'nsteps = 10000\nnstxout = 0\nnstvout = 0\nnstfout = 0\n'
                     'nstlog = 50000\nnstenergy = 50000\nnstxout-compressed = 50000\n'
                     'compressed-x-precision = 10000\nnstlist  = 10\nns_type  = grid\n'
                     'pbc   = xyz\ncoulombtype  = Reaction_field\nrcoulomb_switch = 0.0\n'
                     'rcoulomb  = 1.1\nepsilon_r  = 15\nvdw_type  = cutoff\n'
                     'rvdw_switch  = 0.9\nrvdw   = 1.1\ncutoff-scheme = verlet\n'
                     'coulomb-modifier = Potential-shift\nvdw-modifier  = Potential-shift\n'
                     'epsilon_rf  = 0\nverlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\n'
                     'tc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\nref_t  = 310 310 310\n'
                     'Pcoupl  = berendsen\nPcoupltype  = semiisotropic\ntau_p  = 12.0\n' 
                     'compressibility = 3e-4 3e-4\nref_p  = 1.0 1.0\ngen_vel  = yes\n' 
                     'gen_temp  = 310\ngen_seed  = -1\nconstraints  = none\n'
                     'constraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\n'
                     'lincs_warnangle = 30\nrefcoord_scaling = com')
        gromacs.grompp(f='cgequil.mdp',o=f'EQ{rep}/eq',r=f'CG-system{rep}.pdb',
                       c=f'CG-system{rep}.pdb',maxwarn=-1, n=f'index_sys{rep}.ndx',backup=False)
        os.chdir(f'{subdir}EQ{rep}')
        print("Equillibration starting...")
        gromacs.mdrun(deffnm='eq',backup=False, v=True)
        os.chdir(subdir)
        print("Equillibration complete.")

        # VISUALIZE
        #if rep == 1:
        #    visualise_system(f'EQ{rep}/eq.gro', waters=False, ions=True)

        # SETTING UP A PRODUCTION RUN
        print(f"\nSetting up a production run for CG system {rep}...")
        os.makedirs(f'{subdir}MD{rep}', exist_ok=True)
        os.chdir(subdir)
        with open(f'{subdir}cgmd.mdp','w') as md: # create production .mdp
                    md.write('integrator = md\ntinit = 0.0\ndt = 0.02\nnsteps = 250000000\n'
                            'nstxout = 0\nnstvout = 0\nnstfout = 0\nnstlog = 50000\nnstenergy = 50000\n'
                            'nstxout-compressed = 50000\ncompressed-x-precision = 10000\nnstlist  = 10\n'
                            'ns_type  = grid\npbc   = xyz\ncoulombtype  = Reaction_field\n'
                            'rcoulomb_switch = 0.0\nrcoulomb  = 1.1\nepsilon_r  = 15\n'
                            'vdw_type  = cutoff\nrvdw_switch  = 0.9\nrvdw   = 1.1\n'
                            'cutoff-scheme = verlet\ncoulomb-modifier = Potential-shift\n'
                            'vdw-modifier  = Potential-shift\nepsilon_rf  = 0\n'
                            'verlet-buffer-tolerance = 0.005\ntcoupl  = v-rescale\n'
                            'tc-grps  = PROTEIN LIPID SOL_ION\ntau_t  = 1.0 1.0 1.0\n'
                            f'ref_t  = 310 310 310\nPcoupl  = berendsen\nPcoupltype  = {pcoupltype}\n'
                            f'tau_p  = 12.0\ncompressibility = {compressibility}\nref_p  = {ref_p}\n'
                            'gen_vel  = yes\ngen_temp  = 310\ngen_seed  = -1\nconstraints  = none\n'
                            'constraint_algorithm = Lincs\ncontinuation  = no\nlincs_order  = 4\n'
                            'lincs_warnangle = 30\n')

        gromacs.grompp(f='cgmd.mdp',o=f'MD{rep}/md',c=f'EQ{rep}/eq.gro',
                       maxwarn=-1,n=f'index_sys{rep}.ndx',backup=False)
        #shutil.rmtree('MD', ignore_errors=True)
        print("Production run setup complete.")
        print(f"\n{esc(34, f'CG-system{rep}')} {esc(92, f'complete.')}")

# Setp system
ratio = get_lipid_ratio(pe_pg_ratio=PE_PG_ratio, total=10, card_conc=CARD_conc, tolerance=0.0001)

setup_cg_system(filename1, filename2, Units=Units, rotate_sp=0,
                Repeats=Repeats, Box_Width=Box_Width,Box_Height=Box_Height, Box_Length_ext=Box_Length_ext,
                Embed=Embed,Continuous=False, Ratio=ratio, 
                NamingConvention=f'')

# LOOP EXAMPLE
# concs
#concs = [0.2] #[0, 0.05, 0.1, 0.15]#, 0.2]

#for i in concs:
#    ratio = get_lipid_ratio(pe_pg_ratio=3.5, total=10, card_conc=i, tolerance=0.0001)
#    setup_cg_system(filename1, filename2, Units=4, rotate_sp=180,
#                    Repeats=5, Box_Width=25,Box_Height=21, Box_Length_ext=20,
#                    Embed=2,Continuous=False, Ratio=ratio, 
#                    NamingConvention=f'-c{i}')


In [ ]:
#@title Download Zip File
#%%script false --no-raise-error

def download(nm=None):
    """
    Download results as a zip file.
    """
    if nm == None:
        os.chdir('/content/')
        os.system(f'zip -r {name}.zip {name}')
        files.download(f'{name}.zip')
    else:
        os.chdir('/content/')
        os.system(f'zip -r {name}-{nm}.zip {name}')
        files.download(f'{name}-{nm}.zip')

if Download is True:
    download(f'{Zip_Name}') # usage